In [1]:
using Pkg
Pkg.activate(".")

  Activating new environment at `~/Documents/Github2/Physics-215-Julia/Session 1 - HPC and the Julia Framework/Project.toml`


### KR1: Use ```@code_*``` to examine a simple function. The `*` is replaceable by `native`, `typed`, `warntype`, and others. Discover them.

Let us first define a simple function. Written below is a function that takes two variables and outputs their sum.

In [2]:
function f(x,y) x + y end

f (generic function with 1 method)

In [3]:
@code_native f(1,2)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[2]:1 within `f'
; │┌ @ int.jl:87 within `+'
	leaq	(%rdi,%rsi), %rax
; │└
	retq
	nopw	%cs:(%rax,%rax)
; └


In [4]:
@code_typed f(1.1,2)

CodeInfo(
1 ─ %1 = Base.sitofp(Float64, y)::Float64
│   %2 = Base.add_float(x, %1)::Float64
└──      return %2
) => Float64

In [5]:
@code_warntype f(1.1,2)

Variables
  #self#::Core.Const(f)
  x::Float64
  y::Int64

Body::Float64
1 ─ %1 = (x + y)::Float64
└──      return %1


In [6]:
@code_lowered f(1,2)

CodeInfo(
1 ─ %1 = x + y
└──      return %1
)

In [7]:
@code_llvm f(1,2)

;  @ In[2]:1 within `f'
define i64 @julia_f_1764(i64 signext %0, i64 signext %1) {
top:
; ┌ @ int.jl:87 within `+'
   %2 = add i64 %1, %0
; └
  ret i64 %2
}


Now all of these `macro` functions are new to me so I will just to explain what I understood from our class discussion and some basic googling. These `macro` functions simply convert your `code` into another form of `code` that has a different expression that I believe is more readable by the machine. Using these `@code_*` macros also show us for example, the data types of our inputs, the step-by-step process inside the function and what the output is. In Julia, there are 5 `@code_*` marcos which are `native`, `typed`, `warntype`, `lowered`, and `llvm`. These difference macro functions are I believe just different formms of how the code is being implemented by Julia.

It is also important for us to at least know how to implement these macros for cases when we are unsure of the data types of the processes that are happening inside a function. We can see for example in the code blocks above that the `code_*` marcos return `i64` and `float64` if our inputs are integers or floats for example. It also tells us the process that is being done. For example, in ` %2 = add i64 %1, %0`, we see that the function adds out inputs that are stored or located in `%1` and `%0`.

### KR2: Demonstrate that Julia is able to determine constants in codes.

Let us create a new function `g(x)` that takes a single number then performs a sequence of multiplication and division

In [8]:
function g(x)
    a = sqrt(5) * x
    return exp(2) / a
end

g (generic function with 1 method)

In [9]:
@code_llvm  g(1.1)

;  @ In[8]:1 within `g'
define double @julia_g_1803(double %0) {
top:
;  @ In[8]:2 within `g'
; ┌ @ float.jl:332 within `*'
   %1 = fmul double %0, 0x4001E3779B97F4A8
; └
;  @ In[8]:3 within `g'
; ┌ @ float.jl:335 within `/'
   %2 = fdiv double 0x401D8E64B8D4DDAE, %1
; └
  ret double %2
}


Now the important part of the result of our `@code_llvm` are the following:
```
%1 = fmul double %0, 0x4001E3779B97F4A8
%2 = fdiv double 0x401D8E64B8D4DDAE, %1
```
Now we see that `%1` represents the first line of our function which is `sqrt(5) * x`. We also see that the constant `sqrt(5)` has been stored in a memory location with value `0x4001E3779B97F4A8`. Now what Julia does is if it sees an expression that does not have a variable where it can be simplified into a single constant value, that value will be stored somewhere so that every time that specific value is needed, Julia will just pull the value from the storage instead of computing it again. This can also be seen in the next line where the value of `exp(2)` is now represented by `0x401D8E64B8D4DDAE`.

### KR3: Demonstrate Julia's type-inference and multiple dispatch.

Now before we start creating functions, we must check first if there is already an exisiting Julia function that uses the same name. For example, if we want to create a named `measure` we must first check if it is already existing. We can do this by using the help command in Julia.

In [10]:
?measure

search:

Couldn't find measure
Perhaps you meant merge


No documentation found.

Binding `measure` does not exist.


We see that there are no built-in Julia functions that have the name measure. We also see that Julia gives us possible suggestions of functions that are similar to what we searched. Let us say for example, we plan on making a function with name `exp`, upon checking, we apparently cannot use `exp` as a function name since there is already a pre-exisiting function with that name.

In [11]:
?exp

search: exp exp2 Expr expm1 exp10 export exponent expanduser ExponentialBackOff



```
exp(x)
```

Compute the natural base exponential of `x`, in other words $e^x$.

# Examples

```jldoctest
julia> exp(1.0)
2.718281828459045
```

---

```
exp(A::AbstractMatrix)
```

Compute the matrix exponential of `A`, defined by

$$
e^A = \sum_{n=0}^{\infty} \frac{A^n}{n!}.
$$

For symmetric or Hermitian `A`, an eigendecomposition ([`eigen`](@ref)) is used, otherwise the scaling and squaring algorithm (see [^H05]) is chosen.

[^H05]: Nicholas J. Higham, "The squaring and scaling method for the matrix exponential revisited", SIAM Journal on Matrix Analysis and Applications, 26(4), 2005, 1179-1193. [doi:10.1137/090768539](https://doi.org/10.1137/090768539)

# Examples

```jldoctest
julia> A = Matrix(1.0I, 2, 2)
2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

julia> exp(A)
2×2 Matrix{Float64}:
 2.71828  0.0
 0.0      2.71828
```


Now, let us define again a simple function.

In [12]:
func(x,y) = x + y

func (generic function with 1 method)

Notice that we have created a `generic function with 1 method`. A Julia function can have different ways of implementation and these implementations are called methods. Let us update our `func` function, such that, if both numbers that are inserted are integers, we will take their product instead of sum. To do this, we just write another `method` for our function.

In [13]:
func(x::Int, y::Int) = x * y

func (generic function with 2 methods)

The function has now been updated to have 2 methods, 1 method that takes the sum of any input (assuming of course that its a number) and the second method takes the product if both inputs are integers. Let us test it out.

In [14]:
println(func(1.0, 2.0))
println(func(1.0, 2))
println(func(1, 2.0))
println(func(1, 2))

3.0
3.0
3.0
2


To check all methods for our function `func`, we use the `methods()` function

In [15]:
methods(func)

# 2 methods for generic function "func":
[1] func(x::Int64, y::Int64) in Main at In[13]:1
[2] func(x, y) in Main at In[12]:1

### KR4: Show the difference, if any, between your own sum function `my_sum(x::Vector)` and `@time`. Use a `for`-loop for your customized sum function.

In [16]:
function my_sum(x::Vector)
    s = zero(eltype(x))
    for i in x s = s + i end
    return s
end

my_sum (generic function with 1 method)

The `my_sum()` function only accepts 1D arrays. To make this accept 2D or more matrices, we can simply remove the `::Vector` specification for our `x` variable. Notice also that we used `eltype()` to make sure that the data type of our placeholder for sum is the same as the data type of our list. For example, let us take the sum of the vector below:

In [17]:
trial_vector = [1, 2, 3.1, 4+im]

4-element Vector{ComplexF64}:
 1.0 + 0.0im
 2.0 + 0.0im
 3.1 + 0.0im
 4.0 + 1.0im

Notice that initially out list has different data types to begin with but what Julia did was made them all into Complex numbers to make sure that no data is lost. If for example we have integers and floats in our list, Julia will convert the data types of all elements to float. So getting the sum below, we see that the result is also a complex number, the same as the data type of the elements of our input list.

In [18]:
my_sum(trial_vector)

10.1 + 1.0im

Now let us generate a list of random numbers using the `rand` function and use `@time` to measure the time it takes for our function to get the sum.

In [19]:
x = rand(10_000)
@time my_sum(x)

  0.025157 seconds (8.19 k allocations: 447.076 KiB, 99.83% compilation time)


5039.903223978293

In [20]:
@time my_sum(x)

  0.000021 seconds (1 allocation: 16 bytes)


5039.903223978293

Notice that we have ran the `@time` macro twice. The first use of `@time` compiles the function first and uses a significant amount of time and memory, while the second use of `@time` now has a significant drop in runtime and memory allocation. So note that when profiling and timing Julia codes, we usually disregard the first time running and record the second one instead.

Now let us compare the time it takes for the built-in function `sum` to solve for the sum of all elements of the vector `x`.

In [21]:
@time sum(x)

  0.321681 seconds (76.44 k allocations: 4.518 MiB, 99.81% compilation time)


5039.903223978293

In [22]:
@time sum(x)

  0.000014 seconds (1 allocation: 16 bytes)


5039.903223978293

### KR5: Replicate plotting the Mandelbrot. Use a separate file `Mandelbrot.jl` to contain the function code. Use `include()` function to load the file.